## MONET-Analysis Airnow prep notebook

### How to use

- start notebook and 
- in cell 2 set the start date and end date
- in cell 2 set the filename output (something like AIRNOW_STARTDATE_ENDDATE.nc with STARTDATE and ENDDATE in YYYYMMDD format)

In [1]:
import monetio as mio

/opt/miniconda3/envs/dev/lib/python3.8/site-packages/pyresample/bilinear/__init__.py:50: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


In [16]:
import pandas as pd
filename

dates = pd.date_range(start='2019-08-01',end='2019-08-21',freq='H')

In [17]:
df = mio.airnow.add_data(dates,wide_fmt=False,n_procs=4)

Aggregating AIRNOW files...
Building AIRNOW URLs...
[########################################] | 100% Completed |  3.1s
[########################################] | 100% Completed |  3.2s
[########################################] | 100% Completed |  3.2s
[########################################] | 100% Completed |  3.1s
[########################################] | 100% Completed |  4min 41.0s
[########################################] | 100% Completed |  4min 41.1s
[########################################] | 100% Completed |  4min 41.2s
[########################################] | 100% Completed |  4min 41.3s
    Adding in Meta-data


In [18]:
df = df.dropna(subset=['latitude','longitude'])
df.head()

,time,siteid,site,utcoffset,variable,units,obs,time_local,latitude,longitude,cmsa_name,msa_code,msa_name,state_name,epa_region
0,2019-08-01 00:00:00,000010102,St. John's,-4,OZONE,PPB,2.0,2019-07-31 20:00:00,47.6528,-52.8167,NaN,NaN,NaN,CC,CA
1,2019-08-01 01:00:00,000010102,St. John's,-4,OZONE,PPB,16.0,2019-07-31 21:00:00,47.6528,-52.8167,NaN,NaN,NaN,CC,CA
2,2019-08-01 02:00:00,000010102,St. John's,-4,OZONE,PPB,28.0,2019-07-31 22:00:00,47.6528,-52.8167,NaN,NaN,NaN,CC,CA
3,2019-08-01 03:00:00,000010102,St. John's,-4,OZONE,PPB,31.0,2019-07-31 23:00:00,47.6528,-52.8167,NaN,NaN,NaN,CC,CA
4,2019-08-01 04:00:00,000010102,St. John's,-4,OZONE,PPB,34.0,2019-08-01 00:00:00,47.6528,-52.8167,NaN,NaN,NaN,CC,CA


In [27]:
dfp = df.rename({'siteid':'x'},axis=1).pivot_table(values='obs',index=['time','x'], columns=['variable'])
# dfp

In [28]:
dfx = dfp.to_xarray()
# dfx

In [41]:
# dfplat = df.pivot_table(values='latitude',index=['siteid']).to_xarray()
# dfplon = df.pivot_table(values='longitude',index=['siteid']).to_xarray()
dfpsite = df.rename({'siteid':'x'},axis=1).drop_duplicates(subset=['x'])
dfpsite.head()

,time,x,site,utcoffset,variable,units,obs,time_local,latitude,longitude,cmsa_name,msa_code,msa_name,state_name,epa_region
0,2019-08-01,000010102,St. John's,-4,OZONE,PPB,2.0,2019-07-31 20:00:00,47.65280,-52.81670,NaN,NaN,NaN,CC,CA
481,2019-08-01,000010401,Mount Pearl,-4,OZONE,PPB,30.0,2019-07-31 20:00:00,47.50500,-52.79470,NaN,NaN,NaN,CC,CA
2159,2019-08-01,000010501,Grand Falls Windsor,-4,OZONE,PPB,17.0,2019-07-31 20:00:00,49.01940,-55.80280,NaN,NaN,NaN,CC,CA
2446,2019-08-01,000010601,Goose Bay,-4,OZONE,PPB,21.0,2019-07-31 20:00:00,53.30470,-60.36440,NaN,NaN,NaN,CC,CA
2794,2019-08-01,000010602,MacPherson Avenue -,-4,PM2.5,UG/M3,12.0,2019-07-31 20:00:00,48.95224,-57.92207,NaN,NaN,NaN,CC,CA


In [42]:
test = dfpsite.drop(['time','time_local','variable','obs'],axis=1).set_index('x').dropna(subset=['latitude','longitude']).to_xarray()
test

<xarray.Dataset>
Dimensions:     (x: 3788)
Coordinates:
  * x           (x) object '000010102' '000010401' ... '720210010'
Data variables:
    site        (x) object "St. John's" 'Mount Pearl' ... 'Bayamon 37 (NCore)'
    utcoffset   (x) int64 -4 -4 -4 -4 -4 -4 -4 -4 -4 ... -8 3 -6 -7 -8 -8 -8 -4
    units       (x) object 'PPB' 'PPB' 'PPB' 'PPB' ... 'UG/M3' 'PPB' 'PPB'
    latitude    (x) float64 47.65 47.51 49.02 53.3 ... 38.31 48.17 32.71 18.42
    longitude   (x) float64 -52.82 -52.79 -55.8 -60.36 ... -119.0 -117.1 -66.15
    cmsa_name   (x) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    msa_code    (x) float64 nan nan nan nan nan ... 4.186e+04 nan 4.174e+04 nan
    msa_name    (x) object nan nan ... ' San Diego-Carlsbad-San Marcos, CA ' nan
    state_name  (x) object 'CC' 'CC' 'CC' 'CC' 'CC' ... 'TX' 'CA' nan 'CA' 'PR'
    epa_region  (x) object 'CA' 'CA' 'CA' 'CA' 'CA' ... 'R9' 'R10' 'R9' 'R2'

In [31]:
t = xr.merge([dfx,test])
# t

In [32]:
def get_local_time(ds):
    from numpy import zeros
    if 'utcoffset' in ds.data_vars:
        tim = t.time.copy()
        o = tim.expand_dims({'x':t.x.values}).transpose('time','x')
        on = xr.Dataset({'time_local':o,'utcoffset':t.utcoffset})
        y = on.to_dataframe()
        y['time_local'] = y.time_local + pd.to_timedelta(y.utcoffset, unit='H')
        time_local = y[['time_local']].to_xarray()
        ds = xr.merge([ds,time_local])
    return ds

tt = get_local_time(t)

In [33]:
from util import write_util
tt['siteid'] = (('x'),tt.x.values)
tt['x'] = range(len(tt.x))
t = tt.expand_dims('y').set_coords(['siteid','latitude','longitude']).transpose('time','y','x')
t

<xarray.Dataset>
Dimensions:     (time: 1443, x: 3788, y: 1)
Coordinates:
  * x           (x) int64 0 1 2 3 4 5 6 7 ... 3781 3782 3783 3784 3785 3786 3787
  * time        (time) datetime64[ns] 2019-08-01 ... 2019-08-21T00:30:00
    latitude    (y, x) float64 47.65 47.51 49.02 53.3 ... 36.92 47.93 41.37
    longitude   (y, x) float64 -52.82 -52.79 -55.8 -60.36 ... -94.84 106.9 69.27
    siteid      (y, x) object '000010102' '000010401' ... 'UZB010001'
Dimensions without coordinates: y
Data variables: (12/30)
    BARPR       (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    BC          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    CO          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    NO          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    NO2         (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    NO2Y        (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ...          ...
    cmsa_name   (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    msa_code    (y, x) float64 nan nan nan nan nan ... nan nan 3.306e+04 nan nan
    msa_name    (y, x) object nan nan nan nan nan ... nan ' Miami, OK ' nan nan
    state_name  (y, x) object 'CC' 'CC' 'CC' 'CC' 'CC' ... nan nan nan nan nan
    epa_region  (y, x) object 'CA' 'CA' 'CA' 'CA' ... nan 'R6' 'DSMG' 'DSUZ'
    time_local  (time, y, x) datetime64[ns] 2019-07-31T20:00:00 ... 2019-08-2...

In [34]:
write_util.write_ncf(t,'test5.nc')

Writing: test5.nc
Compressing: BARPR, original_dtype: float64
Compressing: BC, original_dtype: float64
Compressing: CO, original_dtype: float64
Compressing: NO, original_dtype: float64
Compressing: NO2, original_dtype: float64
Compressing: NO2Y, original_dtype: float64
Compressing: NOX, original_dtype: float64
Compressing: NOY, original_dtype: float64
Compressing: OZONE, original_dtype: float64
Compressing: PM10, original_dtype: float64
Compressing: PM2.5, original_dtype: float64
Compressing: PRECIP, original_dtype: float64
Compressing: RHUM, original_dtype: float64
Compressing: RWD, original_dtype: float64
Compressing: RWS, original_dtype: float64
Compressing: SO2, original_dtype: float64
Compressing: SRAD, original_dtype: float64
Compressing: TEMP, original_dtype: float64
Compressing: UV-AETH, original_dtype: float64
Compressing: WD, original_dtype: float64
Compressing: WS, original_dtype: float64
Compressing: cmsa_name, original_dtype: float64
Compressing: msa_code, original_dtype: 

In [35]:
!ls -lh test*

-rw-r--r--  1 barry  staff     0B Apr  8 11:30 test3.nc
-rw-r--r--  1 barry  staff   5.3M Apr  8 13:36 test4.nc
-rw-r--r--  1 barry  staff    28M Apr  8 13:47 test5.nc


In [36]:
ls

Monet-analysis-example.ipynb  reformat_airnow.ipynb
__pycache__/                  test3.nc
control.yaml                  test4.nc
ctest.yaml                    test5.nc
driver.py                     util/
plots/


In [40]:
ls ../data